In [1]:
import os
import model.op
from model.op import FleetSizeOptimizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3


In [2]:
sql_table = pd.DataFrame()
for i in range(20, 70, 10):
    conn = sqlite3.connect(f'input/wind/energy_and_flight_time_{i}_mile_route.sqlite')
    query = """
    SELECT flight_direction, energy_consumption, flight_time, wind_direction_degrees, wind_magnitude_mph
    FROM flight_metrics
    """
    df = pd.read_sql_query(query, conn)
    df['distance'] = i

    sql_table = pd.concat([sql_table, df])  
sql_table = sql_table.reset_index(drop=True)
data_dict = {}
for i in range(0, len(sql_table), 2):
    k1 = sql_table['energy_consumption'][i]
    k2 = sql_table['energy_consumption'][i+1]
    kappa = np.array([[0, k1],[k2, 0]])/160*100

    t1 = sql_table['flight_time'][i]
    t2 = sql_table['flight_time'][i+1]
    tau = np.array([[0, t1],[t2, 0]])+5

    distance = sql_table['distance'][i]
    wind_direction_degrees = sql_table['wind_direction_degrees'][i]
    wind_magnitude_mph = sql_table['wind_magnitude_mph'][i]

    key = (distance, wind_direction_degrees, wind_magnitude_mph)
    data_dict[key] = {'flight_time': tau, 'energy_consumption': kappa}

In [3]:
directory_path = f"output/wind_variation"  # Replace with the path to your directory
file_list = os.listdir(directory_path)
file_names = np.empty(shape=(0, 3))
for i in file_list:
    file_names = np.vstack((file_names, np.array(i.split('_')[:3])))
file_names = file_names.astype(int)

In [44]:
conn = sqlite3.connect('wind_variation_result.sqlite')
for i in range(len(file_names)):
    # Load parameters
    flight_time = data_dict[tuple(file_names[i])]['flight_time']
    energy_consumption = data_dict[tuple(file_names[i])]['energy_consumption']
    flight_distance = np.array([[0, file_names[i][0]], [file_names[i][0], 0]])

    # Call optimizer class and redner summary
    optimizer = FleetSizeOptimizer(flight_time, energy_consumption, schedule='schedule_5min_0612.csv')
    result_path = f'wind_variation/{file_names[i][0]}_{file_names[i][1]}_{file_names[i][2]}_.txt'
    optimizer.parse_result(result_path)
    optimizer.calculate_aircraft_states()
    summary = optimizer.get_summary_statistics(flight_distance, return_summary=True)

    # Save summary statistics
    line_i = np.array([summary['fleet_size'], summary['pads'][0], summary['pads'][1], summary['number_of_repositioning_flights']])
    summary_statistics = pd.DataFrame(line_i.reshape(1,4), columns=['fleet_size', 'pads_at_SFO', 'pads_at_SJC', 'number_of_repositioning_flights'])
    summary_statistics['id'] = f'{file_names[i][0]}_{file_names[i][1]}_{file_names[i][2]}'
    summary_statistics.to_sql('op_summary_statistics', conn, if_exists='append', index=False)

    # Save vertisim inputs
    vertisim_input = pd.concat([optimizer.specificc, optimizer.specificn, optimizer.specificu])
    vertisim_input.reset_index(drop=True, inplace=True)
    vertisim_input['id'] = f'{file_names[i][0]}_{file_names[i][1]}_{file_names[i][2]}'
    vertisim_input.to_sql('vertisim_input', conn, if_exists='append', index=False)



Fleet size: 34.0
Total number of pads: 44; [13 31] 
Total number of flights: 880.0; demand: 771.0; repositioning: 109.0
Total energy consumption: 10560.0 kWh
Total aircraft miles: 52800.0 mi
Total revenue aircraft miles: 46260.0
Ratio of revenue aircraft miles to aircraft miles: 0.8761363636363636
Fleet size: 23.0
Total number of pads: 31; [10 21] 
Total number of flights: 890.0; demand: 771.0; repositioning: 119.0
Total energy consumption: 6230.0 kWh
Total aircraft miles: 26700.0 mi
Total revenue aircraft miles: 23130.0
Ratio of revenue aircraft miles to aircraft miles: 0.8662921348314607
Fleet size: 23.0
Total number of pads: 31; [10 21] 
Total number of flights: 890.0; demand: 771.0; repositioning: 119.0
Total energy consumption: 6230.0 kWh
Total aircraft miles: 26700.0 mi
Total revenue aircraft miles: 23130.0
Ratio of revenue aircraft miles to aircraft miles: 0.8662921348314607
Fleet size: 32.0
Total number of pads: 42; [13 29] 
Total number of flights: 884.0; demand: 771.0; reposi

In [47]:

conn = sqlite3.connect('wind_variation_result.sqlite')
query = """
SELECT *
FROM op_summary_statistics
"""
df = pd.read_sql_query(query, conn)


